# 2. Arrays and linear algebra

## Arrays

Arrays are the standard "container" objects, representing lists/vectors, matrices and higher order tensors.

Arrays can be constructed using the literal syntax:

In [1]:
X = [1,2,3] # a vector

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
M = [1 2 3; 4 5 6; 7 8 9] # matrix: note the different separators

3x3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [9]:
[1 2 3,]

LoadError: LoadError: syntax: unexpected comma in matrix expression
while loading In[9], in expression starting on line 1

Arrays are a parametric type with 2 parameters: 
* the first is the type of the element: this can be either concrete or abstract.
* the second is the number of dimensions.

The literal syntax will attempt to find a "common" type

In [3]:
[1, 2.0, 3]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [4]:
[1, 2.0, 3+4im]

3-element Array{Complex{Float64},1}:
 1.0+0.0im
 2.0+0.0im
 3.0+4.0im

You can also prefix with the desired type:

In [5]:
Any[1,2,3]

3-element Array{Any,1}:
 1
 2
 3

In [6]:
Float64[] # useful for constructing empty arrays

0-element Array{Float64,1}

`Vector` and `Matrix` are aliases for 1- and 2-dimensional `Array`s:

In [7]:
Vector{Float64}

Array{Float64,1}

In [8]:
Matrix{Float64}

Array{Float64,2}

> **Performance note:** there are performance and memory advantages to using Arrays with a concrete type (e.g. `Array{Float64,1}` vs `Array{AbstractFloat,1}` or `Array{Any,1}`), as the compiler is able to infer the type without looking at each element, and able to store them more efficiently.

### Indexing
Indexing is 1-based (like R and Matlab, not 0-based like Python or C), and via square brackets:

In [11]:
X = [5,6,7]
X[3]

7

In [12]:
X[3] = 8 # replace 3rd element
X

3-element Array{Int64,1}:
 5
 6
 8

Matrices are stored in column-major order, and can be indexed by either 2 indices (row, column) or 1 index (raw ordering of the storage)

In [13]:
M = [1 2 3; 4 5 6; 7 8 9]
M[2,1] # 2nd row, 1st column

4

In [14]:
M[3] # 3rd element in column-major storage

7

In [16]:
M[3,:]

1x3 Array{Int64,2}:
 7  8  9

In [17]:
M[3,1:2]

1x2 Array{Int64,2}:
 7  8

In [18]:
X = [1,2,3,4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [20]:
reshape(X,1,4)

1x4 Array{Int64,2}:
 1  2  3  4

### Array operations

Julia distinguishes between functions that are applied to an Array, and functions that are applied to elements of an array.

Operators preceded by a dot (`.`) are applied elementwise:

In [21]:
X = [1,2,3]; Y = [1,4,2]

X == Y # true if and only if all elements are equal

false

In [22]:
X .== Y # compares each element

3-element BitArray{1}:
  true
 false
 false

Same for arithmetic operators:

In [23]:
X .* Y # element-wise multiplication

3-element Array{Int64,1}:
 1
 8
 6

In [24]:
M * Y # matrix multiplication

3-element Array{Int64,1}:
 15
 36
 57

One notable difference to other languages is that `min`/`max` are applied elementwise, and `minimum`/`maximum` are reducing (similar to `+` and `sum`)

In [25]:
min(X,Y)

3-element Array{Int64,1}:
 1
 2
 2

In [18]:
minimum(X)

1

### Other methods for constructing arrays

* `zeros`/`ones` constructs arrays of zeros/ones
* `rand`/`randn` constructs arrays of random U(0,1) or N(0,1) variates
* `eye` constructs identity matrices

In [29]:
zeros(4) # note difference from matlab

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [32]:
rand(3,4)

3x4 Array{Float64,2}:
 0.590506   0.580763   0.297574  0.451487
 0.151956   0.0239065  0.561613  0.991008
 0.0933139  0.380463   0.213581  0.740514

In [34]:
ones(BigInt,3) # optional first argument specifies type

3-element Array{BigInt,1}:
 1
 1
 1

In [22]:
eye(4) # gives matrix, not vector

4x4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

It is also possible to call `Array` directly, though you may get unexpected results:

In [35]:
Array(Float64,3,3)

3x3 Array{Float64,2}:
 2.2103e-314  2.2103e-314  2.2103e-314 
 2.2103e-314  2.2103e-314  2.2103e-314 
 2.2103e-314  2.2103e-314  2.21406e-314

> **Question:**
> 1. Why does this happen?

## Abstract Arrays and Ranges

`Array` is a subtype of `AbstractArray`, which captures all "array-like" objects. The simplest example are the `Range` objects:

In [36]:
R = 0:9 # ranges are specied by start:end

0:9

In [37]:
R = 0:0.1:1 # optional middle element specifies step

0.0:0.1:1.0

In [41]:
R[3] # like arrays we can index

0.2

In [45]:
R[4] = 7 # can't reassign values

LoadError: LoadError: indexed assignment not defined for FloatRange{Float64}
while loading In[45], in expression starting on line 1

In [46]:
2+3*R # can use usual arithmetic operations

2.0:0.3:5.0

Some other `AbstractArray`s:
* `BitArray`: efficient storage of an array of `Bool`s (`true`/`false`)
* `SparseMatrixCSC`: sparse matrix stored in "Compressed Sparse Column" format
* `Diagonal`/`Bidiagonal`/`Tridiagonal`: efficient storage of special matrix types
* `UpperTriangular`/`LowerTriangular`: triangular "mask" over a matrix

In [47]:
UpperTriangular(ones(3,3))

3x3 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  1.0  1.0
 0.0  1.0  1.0
 0.0  0.0  1.0

### Comprehensions

Comprehensions provide convenient and efficient ways to construct arrays from iterable objects.

In [49]:
[x for x in 0:0.1:1]

11-element Array{Float64,1}:
 0.0
 0.1
 0.2
 0.3
 0.4
 0.5
 0.6
 0.7
 0.8
 0.9
 1.0

In [50]:
[exp(x^2/2) for x = -1:0.5:1]

5-element Array{Float64,1}:
 1.64872
 1.13315
 1.0    
 1.13315
 1.64872

> **Performance tip**: this is actually much more efficient than the equivalent vectorised operation (`exp((-1:0.5:1).^2./2)`) as it doesn't require allocating any intermediate arrays.

Comprehensions over multiple objects give arrays:

In [51]:
[i+j for i=1:5, j=1:5]

5x5 Array{Int64,2}:
 2  3  4  5   6
 3  4  5  6   7
 4  5  6  7   8
 5  6  7  8   9
 6  7  8  9  10

In [52]:
[c for c in "αβγ"] # can be used with any iterable object of known length

3-element Array{Char,1}:
 'α'
 'β'
 'γ'

## Tuples

Tuples are small collections of values, and are created by comma-separated values inside regular parantheses `()`:

In [33]:
(1,2) # 2-element tuple

(1,2)

In [34]:
() # empty tuple

()

In [54]:
(1,) # 1-element tuple (note the trailing comma)

(1,)

Tuples are immutable (you can't modify the values or change their length), but unlike arrays there is no performance penalty for using heterogeneous types:

In [55]:
x = (1,2.0,"aa")

(1,2.0,"aa")

In [56]:
x[2] # can be indexed as arrays

2.0

In [57]:
typeof(x)

Tuple{Int64,Float64,ASCIIString}

Tuples are often used as return types when functions return multiple values. 

In [58]:
size(randn(3,4,6)) # returns array dimensions as a tuple

(3,4,6)

In this case, we can *splat* the values:

In [59]:
x,y,z = size(randn(3,4,6))

(3,4,6)

In [60]:
z

6

## Linear Algebra

Basic arithmetic operations:
* `*` for matrix multiplication
* `\` and `/` for matrix division
* `'` is for conjugate transpose (`.'` for non-conjugate version)

In [42]:
M = randn(4,4); x = randn(4)
M' \ x # equivalent to inv(M') * x

4-element Array{Float64,1}:
 -0.38424   
 -0.00329067
  0.107503  
  0.0578994 

Standard factorizations are available:
* `chol`, `qr`, `svd`, `lu`

In [61]:
X = randn(7,4)
Q,R = qr(X)

(
7x4 Array{Float64,2}:
 -0.10389   -0.0136444  -0.652212   -0.15149  
 -0.663802  -0.365414    0.465703    0.215238 
 -0.329307  -0.511423   -0.546004    0.0515446
 -0.263036  -0.178468    0.0741023  -0.477041 
  0.269697  -0.369167   -0.0295726   0.0420129
  0.128996  -0.204607    0.197195   -0.808984 
  0.530628  -0.628278    0.119864    0.210425 ,

4x4 Array{Float64,2}:
 -1.51201  -1.03663   0.684255  -0.335779
  0.0      -2.34072  -1.65708    0.312187
  0.0       0.0      -2.08033   -1.09168 
  0.0       0.0       0.0       -2.42551 )

However you often don't need these.

Julia also has `Factorisation` objects: these perform the factorisation, but keep the object stored in a more efficient format (e.g. the Q matrix is stored as Householder reflectors).

In [62]:
F = qrfact(X)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}}(7x4 Array{Float64,2}:
 -1.51201   -1.03663    0.684255    -0.335779   
  0.60133   -2.34072   -1.65708      0.312187   
  0.298315   0.373821  -2.08033     -1.09168    
  0.238281   0.129101  -0.0710208   -2.42551    
 -0.244315   0.274461   0.253793     0.0503209  
 -0.116856   0.15193    0.00559796   0.589253   
 -0.480689   0.467752   0.35579     -0.000433283,4x4 Array{Float64,2}:
 1.10389  -0.650158   0.565948   0.0125843
 0.0       1.35721   -1.36525   -0.578512 
 0.0       0.0        1.67214    0.136534 
 0.0       0.0        0.0        1.48175  )

In [63]:
F.factors

7x4 Array{Float64,2}:
 -1.51201   -1.03663    0.684255    -0.335779   
  0.60133   -2.34072   -1.65708      0.312187   
  0.298315   0.373821  -2.08033     -1.09168    
  0.238281   0.129101  -0.0710208   -2.42551    
 -0.244315   0.274461   0.253793     0.0503209  
 -0.116856   0.15193    0.00559796   0.589253   
 -0.480689   0.467752   0.35579     -0.000433283

In [64]:
F \ ones(7)

4-element Array{Float64,1}:
 -0.518118 
  1.03513  
 -0.0203734
  0.378598 

These are particularly useful when
* you can exploit the structure of the array: e.g. you know the matrix is symmetric positive definite, so can use `cholfact(M)\x`
* you reuse the same matrix, e.g. perform `qrfact` outside the loop, then reuse the object inside.

By convention, methods ending in a ! mutate their arguments: this is particularly useful for large arrays as it can save memory:

In [65]:
Y=randn(6,2)

6x2 Array{Float64,2}:
 -0.964697   0.385148 
 -0.689135   0.0278044
  0.726723   0.628839 
  0.799998   1.19807  
  0.751188  -0.543631 
 -1.11252   -0.142759 

In [66]:
F = qrfact!(Y)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}}(6x2 Array{Float64,2}:
  2.09181    0.370581 
  0.225465  -1.46918  
 -0.237763   0.423312 
 -0.261736   0.80463  
 -0.245767  -0.361552 
  0.363986  -0.0991238,2x2 Array{Float64,2}:
 1.46118      0.0490323
 2.1754e-314  1.01669  )

In [67]:
Y

6x2 Array{Float64,2}:
  2.09181    0.370581 
  0.225465  -1.46918  
 -0.237763   0.423312 
 -0.261736   0.80463  
 -0.245767  -0.361552 
  0.363986  -0.0991238

Julia's linear algebra routines also work for other numeric types:

In [68]:
X = [1//3 3//5 6//2; 1//2 3//2 8//7; 2//1 9//2 3//8]

3x3 Array{Rational{Int64},2}:
 1//3  3//5  3//1
 1//2  3//2  8//7
 2//1  9//2  3//8

In [69]:
F = lufact(X)

Base.LinAlg.LU{Rational{Int64},Array{Rational{Int64},2}}(3x3 Array{Rational{Int64},2}:
 1//3  3//5     3//1 
 3//2  3//5   -47//14
 6//1  3//2  -705//56,[1,2,3],0)

In [70]:
F \ [1,2,3]

3-element Array{Rational{Int64},1}:
 -393//94
    5//2 
   14//47